## Processing of Non-Rudics data for 17BSITAEPR-2A

Communictations with the prawler ended about two weeks prior to recovery.  Data was recorded on the system but not processed through the RUDICS system or post-processed by the ketch/yawl server.

This notebook will process this data and put it into an archival format (text) in order to be appended to the existing dataset, as well as validate it against the final broadcasted datapoints.

A preview of the transmitted data from this prawler is available: [17BSITAEPR-2A Notebook](ERDDAP_PMEL_erddapy_Prawler_17BSITAEPR2A.ipynb)

__pyversion__==3.6   
__author__==S.Bell

In [1]:
import datetime
print("Last run {0}".format(datetime.datetime.now()))

Last run 2018-09-13 09:01:21.416136


The time of last transmitted profile -  **2017-09-15 04:15:00** and last sequential profileid **17bsitae_p1659**

So all profiles after this will be in the raw downloaded data file without any data post processing.  Additionally, the recording format is such that each instrument spits out a block of data with a headerline and expected number of lines to follow, e.g.

Three instruments on this platform ***CTD, AADI, WETL***

`CTD 09/15/2017 06:08:13  08 035`

CTD - time - 8s sample interval - 35 lines to follow
pressure, temperature, conductivity

AADI (Aandera optode)

WETL (Wetlabs chlor/turb - 695nm,700nm)


In [2]:
import pandas as pd
import numpy as np

In [3]:
file_path='/Volumes/WDC_internal/Users/bell/ecoraid/2017/Moorings/17bsitae/raw/prawler/bering_prawler_lastprofiles.txt'

In [48]:
profile_id = 1658
columns=['datetime','divenumber','press','temp','cond','c1','c2','c3','c4','c5','c6','c7','c8','c9','c695nm','c700nm']
df = pd.DataFrame(columns=columns)

with open(file_path, 'r') as dat:
    for line in dat:
    
        #skip empty or blank lines
        if not line.strip():
            if flagtype == 'WETL':
                #This marks the blank line after a WETL report which should be the end of a dive sample...
                data=np.array([dater,divenumber,press,temp,cond,
                              c1,c2,c3,c4,c5,c6,c7,c8,c9,
                              c695nm,c700nm])
                df_p = pd.DataFrame(data=data.T, columns=columns)
                df_p = df_p.astype({'press':np.float,'temp': np.float,'cond': np.float,'c1': np.float, 'c695nm': np.float, 'c700nm': np.float})
                df = pd.concat([df,df_p])
                continue
            else:
                continue
        
        #initialize blank lists for each new profile.  This assumes well patterned data CTD, AADI, WETL
        if 'CTD' in line:
            profile_id +=1 
            dater = pd.date_range((line.split()[1]+' '+line.split()[2]),periods=int(line.split()[4]),freq='8s')
            divenumber = [profile_id]*int(line.split()[4])
            flagtype='CTD'
            press = temp = cond = []
            continue
        if 'AADI' in line:
            flagtype='AADI'
            c1 = c2 = c3 = c4 = c5 = c6 = c7 = c8 = c9 = []
            continue
        if 'WETL' in line:
            flagtype='WETL'
            c695nm = c700nm = []
            continue

        if flagtype == 'CTD':
            t = line.split()
            press = press+[t[0]]
            temp = temp+[t[1]]
            cond = cond+[t[2]]
        elif flagtype == 'AADI':
            t = line.split()
            c1 = c1 +[t[0]]
            c2 = c2 +[t[1]]
            c3 = c3 +[t[2]]
            c4 = c4 +[t[3]]
            c5 = c5 +[t[4]]
            c6 = c6 +[t[5]]
            c7 = c7 +[t[6]]
            c8 = c8 +[t[7]]
            c9 = c9 +[t[8]]
        elif flagtype == 'WETL':
            t = line.split()
            c695nm = c695nm+[t[1]]
            c700nm = c700nm+[t[3]]


### Apply Calibration Information and Conversions

Conductivity needs to be converted to salinity (TEOS-90), Oxygen needs to be salinity corrected and pressure corrected and percent saturuation calculated, and chlor/fntu needs to be calculated from raw counts

### Salinity/Sigma-T

In [49]:
import seawater

In [50]:
# conductivity to salinity
df['sal'] = seawater.salt(r=10*df['cond']/42.914,t=df['temp'],p=df['press'])

#calculate SigmaT
df['SigmaT'] = seawater.eos80.dens0(t=df['temp'],s=df['sal']) - 1000.

### Oxygen

In [51]:
#!/usr/bin/env python

"""
aanderaa_corrO2_sal.py

calculate salinity (and pressure) corrected oxygen concenration due to salinity.

Most relevant for Aanderaa oxygen optodes that have salinity set at 0 (fresh water) for O2 calculations

2018-04-25 S.Bell: CORRECTED.  Use dens insted of pden and dens0 for general corrections.
    (should be pretty small correction). (maybe potential density is more appropiate but I
    can't find the reference)
  Make sure proper variables are being passed into TEOS routines


"""
#System Stack
import datetime

import numpy as np
import seawater as sw

__author__   = 'Shaun Bell'
__email__    = 'shaun.bell@noaa.gov'
__created__  = datetime.datetime(2016, 11, 1)
__modified__ = datetime.datetime(2016, 11, 1)
__version__  = "0.1.0"
__status__   = "Development"
__keywords__ = 'oxygen','correction'


"""------------------------------------- Shear Calc -----------------------------------"""

def O2_sal_comp(oxygen_conc=None,salinity=None,temperature=None,internal_sal=0.0):
    """
    From Aandera April2007 -TD 218 Operating Manual - Oxygen Optodes (pg32)

    S - salinity in ppt
    Ts - scaled temperature
        = ln((298.15 - temperature)/(273.15 + temperature))
    B,C coefs

    """

    coefs = {'B0': -6.24097e-3,
             'B1': -6.93498e-3,
             'B2': -6.90358e-3,
             'B3': -4.29155e-3,
             'C0': -3.11680e-7}

    scaled_temp = np.log((298.15 - temperature)/(273.15 + temperature))

    exp_a = (salinity - internal_sal)
    exp_b = (coefs['B0'] + coefs['B1']*scaled_temp + coefs['B2']*scaled_temp**2. + coefs['B3']*scaled_temp**3.)
    exp_c = coefs['C0'] * (salinity**2. - internal_sal**2.)

    return oxygen_conc * np.exp(exp_a*exp_b + exp_c)

def O2_dep_comp(oxygen_conc=None,depth=None):
    """
    From Aandera Operating Manual - Oxygen Optodes

    Small correction for normal FOCI operations (<500m) but should be used for deep stations
    """
    return oxygen_conc * (1.0+ (0.032 * depth)/1000.)

def O2PercentSat(oxygen_conc=None, temperature=None, salinity=None, pressure=0):
    """
    # calculate oxygen saturation
    # Garcia and Gorden 1992 - from Seabird Derived Parameter Formulas
    
    """

    coefs = {'GG_A0': 2.00907,
             'GG_A1': 3.22014,
             'GG_A2': 4.0501,
             'GG_A3': 4.94457,
             'GG_A4': -0.256847,
             'GG_A5': 3.88767,
             'GG_B0': -0.00624523,
             'GG_B1': -0.00737614,
             'GG_B2': -0.010341,
             'GG_B3': -0.00817083,
             'GG_C0': -0.000000488682}

    scaled_temp = np.log((298.15 - temperature)/(273.15 + temperature))

    Oxsol_pri = np.exp(coefs['GG_A0'] + coefs['GG_A1'] * scaled_temp + coefs['GG_A2'] * (scaled_temp) ** 2 + \
                coefs['GG_A3'] * (scaled_temp) ** 3 + coefs['GG_A4'] * (scaled_temp) ** 4 + \
                coefs['GG_A5'] * (scaled_temp) ** 5 + salinity * (coefs['GG_B0'] + coefs['GG_B1'] * scaled_temp +\
                coefs['GG_B2'] * (scaled_temp) ** 2 + coefs['GG_B3'] * (scaled_temp) ** 3) + coefs['GG_C0'] * (salinity) ** 2)

    #determine sigmatheta and convert Oxygen from micromoles/kg to ml/l
    #calculate new oxygen saturation percent using derived oxsol
    sigmatheta_pri = sw.eos80.pden(salinity, temperature, pressure)
    OxPerSat_pri = ( (oxygen_conc * sigmatheta_pri / 44660) / Oxsol_pri ) * 100.
         
    #replace nan/1e35 with 1e35, >1e10
    try:
        OxPerSat_pri[oxygen_conc >= 1e30] = np.nan
    except:
        if OxPerSat_pri >= 1e30:
            OxPerSat_pri = np.nan
            
    
    return OxPerSat_pri

def O2_molar2umkg(oxygen_conc=None,salinity=None,temperature=None,pressure=None):
    """unit conversalinity=Noneion for micromole/liter -> micromole/kg"""
    sigmatheta_pri = sw.eos80.dens(s=salinity, t=temperature, p=pressure)
    density = (sigmatheta_pri / 1000)
    oxygen_conc = oxygen_conc / density


    return oxygen_conc

In [52]:
# apply salinity and depth corrections to oxygen optode and recalc percentsat
O2_corr = O2_dep_comp(oxygen_conc=df['c1'], depth=df['press'])
O2_corr = O2_sal_comp(oxygen_conc=O2_corr, salinity=df['sal'], temperature=df['temp'])
df['DO'] = O2_molar2umkg(oxygen_conc=O2_corr,
                                 salinity=df['sal'],
                                 temperature=df['temp'],
                                 pressure=df['press'])
df['DO_Sat'] = O2PercentSat(oxygen_conc=O2_corr, 
                                 salinity=df['sal'],
                                 temperature=df['temp'],
                                 pressure=df['press'])

### Chlor and FNTU

In [53]:
df['Chlor'] = 0.0251 * (df['c695nm'] - 61) #ug/l
df['Turb'] = 0.2109 * (df['c700nm'] - 51) #NTU


In [54]:
df.drop(['c1','c3','c4','c5','c6','c7','c8','c9','c695nm','c700nm'], axis=1, inplace=True)

In [55]:
df.to_csv('processed.txt')

In [56]:
dfg = df.groupby('divenumber')

In [57]:
dfg.get_group(1659)

,datetime,divenumber,press,temp,cond,c2,sal,SigmaT,DO,DO_Sat,Chlor,Turb
0,2017-09-15 04:09:12,1659,4.050000,9.1562,3.35284,9.207,31.094178,24.037369,284.844817,101.395260,3.5140,1.0545
1,2017-09-15 04:09:20,1659,3.730000,9.1967,3.39984,9.213,31.542221,24.381351,283.845105,101.491101,3.4638,0.4218
2,2017-09-15 04:09:28,1659,4.290000,9.2112,3.40144,9.215,31.545622,24.381757,283.935896,101.558777,3.4387,1.0545
3,2017-09-15 04:09:36,1659,4.590000,9.1965,3.40099,9.206,31.553860,24.390480,283.938823,101.533985,3.5140,0.6327
4,2017-09-15 04:09:44,1659,4.830000,9.1897,3.40051,9.194,31.554829,24.392294,284.000717,101.541906,3.4387,1.0545
5,2017-09-15 04:09:52,1659,5.330000,9.1872,3.40041,9.190,31.555794,24.393436,284.054837,101.556710,3.4387,0.6327
6,2017-09-15 04:10:00,1659,6.370000,9.1589,3.39797,9.171,31.555255,24.397402,284.127299,101.519758,3.5140,0.4218
7,2017-09-15 04:10:08,1659,7.160000,9.1320,3.39614,9.141,31.559867,24.405170,284.329102,101.536138,4.0160,0.6327
8,2017-09-15 04:10:16,1659,7.710000,9.1246,3.39551,9.136,31.559686,24.406172,284.350015,101.527252,3.7650,1.0545
9,2017-09-15 04:10:24,1659,8.250000,9.1189,3.39504,9.126,31.559652,24.407027,284.496579,101.567118,3.8905,0.2109
